In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque
from tqdm import tqdm
import pandas as pd
import copy

In [2]:
def P_0_matrix(x):
    P_0_matrix=np.array([[1-x,x,0,0,0],
                   [1-x,0,x,0,0],
                   [1-x,0,0,x,0],
                   [1-x,0,0,0,x],
                   [1-x,0,0,0,x]])  
    return P_0_matrix

In [3]:
P_1_matrix=np.array([[1,0,0,0,0],
                   [1,0,0,0,0],
                   [1,0,0,0,0],
                   [1,0,0,0,0],
                   [1,0,0,0,0]])
P_1_matrix

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [4]:
def reward_matrix(x):
    reward_matrix=[[x**1,0],[x**2,0],[x**3,0],[x**4,0],[x**5,0]]
    return reward_matrix

In [5]:
action_space=[0,1]
state_space=[0,1,2,3,4]
classes=2
N=15 # 10 of class 1 and 5 of class 2
M=6
epsilon=0.1
subsidy=0
arm_indexes=[i for i in range(N)]
gamma=0.99
prob=M/N

In [6]:
def step(state,state_space,action,P_0_matrix,P_1_matrix,reward_matrix):
  new_state=state
  if(action==1):
    new_state=np.random.choice(state_space,replace=True,p=P_1_matrix[state])
    reward=reward_matrix[state][1]
  else:
    new_state=np.random.choice(state_space,replace=True,p=P_0_matrix[state])
    reward=reward_matrix[state][0]
  return [new_state,reward]

In [7]:
Q_values=[]
for i in range(0,classes):
  Q_values.append(reward_matrix((i+0.8)/10.0))
  
Q_values

[[[0.1, 0],
  [0.010000000000000002, 0],
  [0.0010000000000000002, 0],
  [0.00010000000000000002, 0],
  [1.0000000000000003e-05, 0]],
 [[0.2, 0],
  [0.04000000000000001, 0],
  [0.008000000000000002, 0],
  [0.0016000000000000003, 0],
  [0.0003200000000000001, 0]]]

In [10]:
def Q_value_update(Q_values,state,next_state,action,reward,arm_index,subsidy,alpha):
  if action==1:
    Q_values[arm_index][state][action]=(1-alpha)*(Q_values[arm_index][state][action])+alpha*(reward+max(Q_values[arm_index][next_state])-(np.array(Q_values[arm_index]).sum())/10)
    return Q_values

  else:
    Q_values[arm_index][state][action]=(1-alpha)*(Q_values[arm_index][state][action])+alpha*(reward+subsidy+max(Q_values[arm_index][next_state])-(np.array(Q_values[arm_index]).sum())/10)
    return Q_values




In [11]:
def act(Q_values,epsilon,state,arm_index):
  decision=np.random.binomial(n=1,p=epsilon,size=1)
  if decision==1:
    action=random.choice([0,1])
  else:
    action=np.argmax(Q_values[arm_index][state])
  return action

In [12]:
alternative_indexes=[0 for _ in range(N)]
current_state=[0 for _ in range(classes)]  

In [15]:
nu=np.zeros((N,len(state_space),len(action_space)))


In [14]:
rewards=[]
subsidies=[]
action_avgs=[]
betas=[]
alphas=[]
epsilon=0.1


In [16]:
def alpha(i):
    return 1/np.ceil(1+(i/10000))

In [ ]:
reward_sum=0
action_sum=[0 for _ in range(classes)]
total_action_sum=0
for i in range(1,100000001):
    # top_M_indices=act(alternative_indexes,N,M,epsilon)
    # alpha=1/(1+(i/500))
    if i%50==0:
        beta=1/(1+np.ceil(i*np.log(i)/10000))
    else:
      beta=0
    action_sum_flag=0

    for index in range(classes):
        action=act(Q_values,epsilon,current_state[index],index)
        action_sum[index]+=action


        x=step(current_state[index],state_space,action,P_0_matrix((index+0.8)/10.0),P_1_matrix,reward_matrix((index+0.8)/10.0))
        new_state=x[0]
        reward=x[1]

        Q_value_update(Q_values,current_state[index],new_state,action,reward,index,subsidy,alpha(nu[index][current_state[index],action]))
        nu[index][current_state[index],action]=nu[index][current_state[index],action]+1
        current_state[index]=new_state
        reward_sum+=reward
        
        #print(x)
    # print(i)
    #print(reward_sum)
    total_action_sum=sum(action_sum)
    subsidy+=beta*(2*action_sum[0]/(3*i)+action_sum[0]/(3*i)-prob)
    subsidies.append(subsidy)
    # epsilon=epsilon*gamma
    # if epsilon<=0.01:
    #     epsilon=0.01
    rewards.append(reward_sum/i)
    action_avgs.append(total_action_sum/i)
    # betas.append(beta)






